This notebook takes a list of route ids and calls the Mountain Project api to acquire the route information.

In [9]:
from pymongo import MongoClient
import json, re, requests, pprint, urllib, time
from bs4 import BeautifulSoup
from mp_api import MP
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
key_path = '../data/mp_api_key.txt'
with open(key_path, 'r') as key_text:
    key = key_text.read()

In [33]:
routes_path = '../data/route_ids.txt'
with open(routes_path, 'r') as routes_text:
    route_ids = routes_text.read().splitlines()
print(len(route_ids))

4982


In [34]:
client = MongoClient('localhost', 27017)
db = 'mountain_project'
collection = 'routes'
if collection in client[db].list_collection_names():
    (client[db])[collection].drop()

regional_routes = (client[db])['regional_routes']

for region in regional_routes.find():
    routes = region['data']
    for route in routes:
        (client[db])[collection].insert_one(route)

In [35]:
(client[db])[collection].count_documents({})

4982

In [36]:
new_routes_path = '../data/new_route_ids.txt'
with open(new_routes_path, 'r') as new_routes_text:
    route_ids = new_routes_text.read().splitlines()
print(len(route_ids))

75614


In [ ]:
client = MongoClient('localhost', 27017)
db = 'mountain_project'
collection = 'routes'

route_db = MP(key, client, db, collection)

for idx in range(100, len(route_ids), 100):
    try:
        route_db.api_get_routes(route_ids[idx - 100: idx])
    except:
        print('broken api call, idx: ', idx)
    last = idx
route_db.api_get_routes(route_ids[last:])

In [41]:
route_db.collection.count_documents({})

80495

In [51]:
pprint.pprint(route_db.collection.find_one({'name': 'Skywalker'}))

{'_id': ObjectId('5e8ce16965c4c9cf4600bd91'),
 'id': 107198282,
 'imgMedium': 'https://cdn-files.apstatic.com/climb/108359801_medium_1494278622.jpg',
 'imgSmall': 'https://cdn-files.apstatic.com/climb/108359801_small_1494278622.jpg',
 'imgSmallMed': 'https://cdn-files.apstatic.com/climb/108359801_smallMed_1494278622.jpg',
 'imgSqSmall': 'https://cdn-files.apstatic.com/climb/108359801_sqsmall_1494278622.jpg',
 'latitude': 49.6693,
 'location': ['International',
              'North America',
              'Canada',
              'British Columbia',
              'Squamish',
              'Shannon Falls',
              'Shannon Falls Wall'],
 'longitude': -123.1583,
 'name': 'Skywalker',
 'pitches': 5,
 'rating': '5.8',
 'starVotes': 473,
 'stars': 4.5,
 'type': 'Trad',
 'url': 'https://www.mountainproject.com/route/107198282/skywalker'}
